# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to Gemeni API
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

# Install Required Packages

In [4]:
!pip install -qU langchain langchain-google-genai google-generativeai
# Note: The Python SDK for the Gemini API, is contained in the google-generativeai package, in order to use Gemini API Directly.


# Get your [Gemini API Key](https://makersuite.google.com/app/apikey)

In [5]:
# Langchain version
!pip show langchain

Name: langchain
Version: 0.3.1
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Method 01: Using API key for auth

In [6]:
# Getting Key from ENVs
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

## Method 02: Using JSON file for Auth
### Complete Procedure can be learn from [here](https://github.com/panaversity/learn-applied-generative-ai-fundamentals/blob/main/21_langchain_ecosystem/langchain/00_gemini_langchain/LangChain_with_Gemini.ipynb), look for **Method 02: Using JSON file for Auth**

In [7]:
# Import the os module to interact with the operating system
import os

# Set the environment variable for Google Cloud application credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Sec/fresh-key.json"


# API calls to Gemini API

## Chat API : Gemini

Let's start with a direct API calls to Gemini.

In [12]:
import google.generativeai as genai

# Set the API key for Google Generative AI
genai.configure(api_key=GOOGLE_API_KEY)

llm_model = genai.GenerativeModel("gemini-1.5-flash")


In [13]:
def get_completion(prompt: str, model= llm_model) -> str:
    """
    Generate a completion for the given prompt using the specified language model.

    Args:
    prompt (str): The input prompt to complete.
    model (ll_model): The language model to use.

    Returns:
    str: The completed text.
    """

    response = model.generate_content(prompt)
    return response.text

In [14]:
get_completion("What is 1+1?")

'1 + 1 = 2 \n'

In [15]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [16]:
style = """American English \
in a calm and respectful tone
"""

In [17]:
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [18]:
response = get_completion(prompt)

In [19]:
response

"I am quite frustrated that the lid of my blender came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I would really appreciate your help with this. \n"

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
# langchain_google_genai: Provides integration with Google's Generative AI


# MODEL_NAME: Specify the Generative AI model to use
MODEL_NAME = "gemini-1.5-flash"  # Gemini 1.5 Flash model
# TEMPERATURE: Control the randomness of model responses (0 = deterministic, 1 = very random)
TEMPERATURE = 0.0  # Balanced randomness and determinism

def initialize_llm() -> ChatGoogleGenerativeAI:
    """
    Initialize an instance of ChatGoogleGenerativeAI with specified parameters.

    Returns:
    ChatGoogleGenerativeAI: The initialized language model.
    """
    # Initialize ChatGoogleGenerativeAI with model name and temperature
    return ChatGoogleGenerativeAI(
        model=MODEL_NAME,  # Generative AI model name
        temperature=TEMPERATURE,  # Response randomness
        # No API key required when using .json file authentication
        api_key=GOOGLE_API_KEY,  # Google API key for authentication
    )

# Initialize the LLaM instance
llm: ChatGoogleGenerativeAI = initialize_llm()  # ChatGoogleGenerativeAI instance
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7b80c21335e0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x7b80c1f7a3e0>, default_metadata=())

## Prompt template

**Prompt templates** in LangChain are predefined structures that help generate prompts for language models. They ensure that the prompts are consistent and tailored to specific tasks, which can improve the quality of the AI's responses.

### Key Features of `ChatPromptTemplate`
1. **Template Creation**:
   - You can create a template that includes placeholders for dynamic content. This allows you to define a structure for your prompts, specifying where user inputs and AI responses should go.

2. **Dynamic Formatting**:
   - At runtime, you can fill in the placeholders with actual values. This means you can generate prompts on-the-fly based on the current context of the conversation.

3. **Style and Tone Customization**:
   - You can specify the style and tone for the AI's responses. For example, you might want the AI to respond in a calm and respectful tone or even in a specific dialect or language.

4. **Message Formatting**:
   - The template can format messages according to the specified style and content. This ensures that the AI's responses are coherent and contextually appropriate.

### Practical Use Cases
- **Customer Support**: Helps in creating dynamic and contextually relevant prompts for customer interactions, ensuring that the AI provides helpful and polite responses.
- **Chatbots**: Enhances the chatbot's ability to maintain a consistent tone and style, making interactions more engaging and user-friendly.
- **Personal Assistants**: Enables virtual assistants to maintain a coherent and engaging conversation with users, improving the overall user experience.

### Example Scenario
Imagine you have a customer who is upset about a blender malfunction. Using a `ChatPromptTemplate`, you can create a prompt that captures the customer's complaint in a specific tone and language. The AI can then generate a response that addresses the issue in a polite and contextually appropriate manner. This ensures that the conversation remains professional and helpful, even when dealing with frustrated customers.

In [21]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [22]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [23]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [24]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [25]:
customer_style = """American English \
in a calm and respectful tone
"""

In [26]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [27]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [28]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [29]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [30]:
# Call the LLM to translate to the style of the customer message
customer_response = llm.invoke(customer_messages)

In [31]:
print(customer_response.content)

I'm quite frustrated that my blender lid came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now. 



In [32]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [33]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [34]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [35]:
service_response = llm.invoke(service_messages)
print(service_response.content)

```
Ahoy there, matey!  The warranty, ye see, don't cover the cost o' cleanin' yer kitchen.  It seems ye be the one who made a mistake, forgettin' to put the lid on yer blender before startin' it.  Avast, me hearties!  That's a bit o' bad luck, but we be sailin' on.  See ye later!
``` 



## Output Parsers

Sure! Let's break down the concept of **output parsers** in LangChain, focusing on their purpose and functionality, without using any code examples.

### What are Output Parsers?
Output parsers in LangChain are tools designed to transform the raw text responses from language models into more structured and useful formats. They help in extracting specific information from the model's output, making it easier to work with and integrate into applications.

### Key Features of Output Parsers
1. **Structured Data Extraction**:
   - Output parsers can extract specific pieces of information from a text, such as whether an item was purchased as a gift, the number of delivery days, or sentences about the price. This is particularly useful for applications that need to process and analyze structured data.

2. **Schema Definition**:
   - You can define schemas that specify the type of information to be extracted. For example, you might define a schema to extract whether an item was a gift, the delivery time, and comments on the price. This ensures that the extracted data is consistent and follows a predefined structure.

3. **Format Instructions**:
   - Output parsers provide format instructions that guide the language model on how to structure its output. This helps in generating responses that are easy to parse and integrate into other systems.

### Practical Use Cases
- **Customer Reviews**: Extracting key information from customer reviews, such as delivery times, gift status, and price comments, to analyze customer feedback and improve products or services.
- **Chatbots**: Enhancing chatbot interactions by extracting relevant details from user inputs, making the responses more contextually appropriate and useful.
- **Data Analysis**: Converting unstructured text data into structured formats for easier analysis and integration with databases or other analytical tools.

### Example Scenario
Imagine you have a customer review for a product, and you want to extract specific details such as whether the product was a gift, how long it took to be delivered, and any comments about its price. An output parser can take the raw review text and transform it into a structured format, such as a JSON object, with clearly defined fields for each piece of information. This structured data can then be easily used for further analysis or reporting.

In [36]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [37]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [38]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [39]:
messages = prompt_template.format_messages(text=customer_review)
response = llm.invoke(messages)
print(response.content)

```json
{
"gift": true,
"delivery_days": 2,
"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [40]:
type(response.content)

str

In [41]:
# You will get an error by running this line of code
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [42]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [43]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [44]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [45]:
format_instructions = output_parser.get_format_instructions()

In [46]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [47]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [48]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [49]:
response = llm.invoke(messages)

In [50]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [51]:
output_dict = output_parser.parse(response.content)

In [52]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [53]:
type(output_dict)

dict

In [54]:
output_dict.get('delivery_days')

'2'

In [55]:
output_dict.get('price_value')

"It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."